In [3]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT,GPSConfig,ResultConfig
import pandas as pd

In [4]:
#load network data and graph
network = Network("../osmnx_example/beijing/edges.shp","fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

[2025-02-21 17:32:43.804] [info] [network.cpp:72] Read network from file ../osmnx_example/beijing/edges.shp
Nodes 157738 edges 393216
[2025-02-21 17:32:45.909] [info] [network.cpp:172] Number of edges 393216 nodes 157738
[2025-02-21 17:32:45.909] [info] [network.cpp:173] Field index: id 18 source 0 target 1
[2025-02-21 17:32:46.246] [info] [network.cpp:176] Read network done.
[2025-02-21 17:32:46.247] [info] [network_graph.cpp:17] Construct graph from network edges start
[2025-02-21 17:32:46.295] [info] [network_graph.cpp:30] Graph nodes 157738 edges 393216
[2025-02-21 17:32:46.295] [info] [network_graph.cpp:31] Construct graph from network edges end


### Precompute an UBODT file

**This step can be skipped if you already created one UBODT file.**

In [ ]:
from fmm import UBODTGenAlgorithm
ubodt_gen = UBODTGenAlgorithm(network,graph)
status = ubodt_gen.generate_ubodt("../osmnx_example/beijing/ubodt.txt", 4, binary=False, use_omp=True)
print(status)

[2025-02-21 12:06:36.625] [info] [ubodt_gen_algorithm.cpp:76] Start to generate UBODT with delta 4
[2025-02-21 12:06:36.625] [info] [ubodt_gen_algorithm.cpp:77] Output format csv
Status: success
Time takes 0.82 seconds

[2025-02-21 12:06:36.708] [info] [ubodt_gen_algorithm.cpp:105] Progress 64 / 649
[2025-02-21 12:06:36.787] [info] [ubodt_gen_algorithm.cpp:105] Progress 128 / 649
[2025-02-21 12:06:36.866] [info] [ubodt_gen_algorithm.cpp:105] Progress 192 / 649
[2025-02-21 12:06:36.951] [info] [ubodt_gen_algorithm.cpp:105] Progress 256 / 649
[2025-02-21 12:06:37.046] [info] [ubodt_gen_algorithm.cpp:105] Progress 320 / 649
[2025-02-21 12:06:37.125] [info] [ubodt_gen_algorithm.cpp:105] Progress 384 / 649
[2025-02-21 12:06:37.199] [info] [ubodt_gen_algorithm.cpp:105] Progress 448 / 649
[2025-02-21 12:06:37.278] [info] [ubodt_gen_algorithm.cpp:105] Progress 512 / 649
[2025-02-21 12:06:37.370] [info] [ubodt_gen_algorithm.cpp:105] Progress 576 / 649
[2025-02-21 12:06:37.441] [info] [ubodt_gen

### FMM model creation and map matching

In [ ]:
ubodt = UBODT.read_ubodt_csv("../osmnx_example/beijing/ubodt.txt")
model = FastMapMatch(network,graph,ubodt)

# configuration parameters
k = 4
radius = 0.4
gps_error = 0.5
fmm_config = FastMapMatchConfig(k,radius,gps_error)

input_config = GPSConfig()
input_config.file = "../osmnx_example/beijing/trips.csv"
input_config.id = "id"

result_config = ResultConfig()
result_config.file = "../osmnx_example/beijing/mr.txt"
result_config.output_config.write_opath = True
print(result_config.to_string())

matched_df = pd.read_csv("../osmnx_example/beijing/mr.txt", delimiter=";")
print(matched_df.head())


NameError: name 'UBODT' is not defined

[2025-02-21 17:32:55.837] [info] [ubodt.cpp:208] Reading UBODT file (CSV format) from ../osmnx_example/beijing/ubodt.txt
[2025-02-21 17:32:56.857] [info] [ubodt.cpp:236] Read rows 1000000
[2025-02-21 17:32:57.680] [info] [ubodt.cpp:236] Read rows 2000000
[2025-02-21 17:32:58.490] [info] [ubodt.cpp:236] Read rows 3000000
[2025-02-21 17:32:59.390] [info] [ubodt.cpp:236] Read rows 4000000
[2025-02-21 17:33:00.226] [info] [ubodt.cpp:236] Read rows 5000000
[2025-02-21 17:33:01.232] [info] [ubodt.cpp:236] Read rows 6000000
[2025-02-21 17:33:02.010] [info] [ubodt.cpp:236] Read rows 7000000
[2025-02-21 17:33:02.777] [info] [ubodt.cpp:236] Read rows 8000000
[2025-02-21 17:33:03.745] [info] [ubodt.cpp:236] Read rows 9000000
[2025-02-21 17:33:04.657] [info] [ubodt.cpp:236] Read rows 10000000
[2025-02-21 17:33:05.574] [info] [ubodt.cpp:236] Read rows 11000000
[2025-02-21 17:33:06.313] [info] [ubodt.cpp:236] Read rows 12000000
[2025-02-21 17:33:07.153] [info] [ubodt.cpp:236] Read rows 13000000
[202

KeyboardInterrupt: 

In [7]:
from shapely.wkt import loads
from shapely.geometry import mapping
import pandas as pd
import json
from ipyleaflet import Map, GeoJSON, WidgetControl
import ipywidgets as widgets

# Load matched and original data
matched_df = pd.read_csv("../osmnx_example/stockholm/mr.txt", delimiter=";")
original_df = pd.read_csv("../osmnx_example/stockholm/trips.csv", delimiter=";")

# Merge original and matched trajectories by 'id'
merged_df = pd.merge(matched_df[['id', 'mgeom']], original_df[['id', 'geom']], on='id')

# Initialize lists for GeoJSON
original_geojson_list = []
matched_geojson_list = []

for _, row in merged_df.iterrows():
    # Convert original and matched geometry to GeoJSON
    original_geom = loads(row["geom"])  # 'geom' is the column for original trajectories
    matched_geom = loads(row["mgeom"])  # 'mgeom' is the column for matched trajectories
    
    original_geojson_list.append({
        "type": "Feature",
        "geometry": mapping(original_geom),
        "properties": {"id": row["id"], "type": "Original"}
    })
    
    matched_geojson_list.append({
        "type": "Feature",
        "geometry": mapping(matched_geom),
        "properties": {"id": row["id"], "type": "Matched"}
    })

# Create GeoJSON FeatureCollection for both original and matched
geojson_data = {"type": "FeatureCollection", "features": original_geojson_list + matched_geojson_list}

# Save to file (optional)
with open("merged_trajectories.geojson", "w") as f:
    json.dump(geojson_data, f)

# Initialize the map
m = Map(center=(59.34, 18.06), zoom=13)

# Add both original and matched layers with different colors
original_layer = GeoJSON(name="Original Trajectories", data={"type": "FeatureCollection", "features": original_geojson_list},
                         style={"color": "blue", "weight": 3})
matched_layer = GeoJSON(name="Matched Trajectories", data={"type": "FeatureCollection", "features": matched_geojson_list},
                        style={"color": "orange", "weight": 3})

m.add_layer(original_layer)
m.add_layer(matched_layer)

# Display the map
m


Map(center=[59.34, 18.06], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

### Map Matching after obfuscation